# Direct and inverse geometry of 3d robots
This notebook introduces the kinematic tree of Pinocchio for a serial manipulator, explain how to compute the forward and inverse geometry (from configuration to end-effector placements, and inversely). The ideas are examplified with a simplified case-study taken from parallel robotics.

**Important: to make sure your repository is easily updated from git shall it be needed, create a copy of this notebook before working on it.**

In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need Pinocchio, meshcat and SciPy for the solvers

In [2]:
import time
import math
import numpy as np
from numpy.linalg import norm
import pinocchio as pin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs

## Kinematic tree in Pinocchio
Let's now play with 3D robots. We will load the models from URDF files.

*The robot UR5* is a low-cost manipulator robot with good performances. It is a fixed robot with one 6-DOF arms developed by the Danish company Universal Robot. All its 6 joints are revolute joints. Its configuration is in R^6 and is not subject to any constraint. The model of UR5 is described in a URDF file, with the visuals of the bodies of the robot being described as meshed (i.e. polygon soups) using the Collada format ".dae". Both the URDF and the DAE files are available in the repository in the model directory. 

This robot model, as well as other models used in the notebooks, are installed from the apt paquet robotpkg-example-robot-data and stored in /opt/openrobots/share/example-robot-data.


In [ ]:
# %load lab_0/generated/simple_pick_and_place_1
robot = robex.load("ur5")


The kinematic tree is represented by two C++ objects called Model (which contains the model constants: lengths, masses, names, etc) and Data (which contains the working memory used by the model algorithms). Both C\++ objects are contained in a unique Python class. The first class is called RobotWrapper and is generic.

In [5]:
print(robot.model)

Nb joints = 7 (nq=6,nv=6)
  Joint 0 universe: parent=0
  Joint 1 shoulder_pan_joint: parent=0
  Joint 2 shoulder_lift_joint: parent=1
  Joint 3 elbow_joint: parent=2
  Joint 4 wrist_1_joint: parent=3
  Joint 5 wrist_2_joint: parent=4
  Joint 6 wrist_3_joint: parent=5



For the next steps, we are going to work with the RobotWrapper.

Import the class RobotWrapper and create an instance of this class in the python terminal. At initialization, RobotWrapper will read the model description in the URDF file given as argument. In the following, we will use the model of the UR5 robot, available in the directory "models" of pinocchio (available in the homedir of the VBox). The code of the RobotWrapper class is in /opt/openrobots/lib/python2.7/site-packages/pinocchio/robot_wrapper.py . Do not hesitate to have a look at it and to take inspiration from the implementation of the class functions.

Here are some import methods of the class.


* robot.q0 contains a reference initial configuration of the robot (not a pretty good one for the UR-5).

* robot.index('joint name') returns the index of the joint.

In [6]:
robot.index(' wrist_3_joint')

7

* robot.model.names is a container (~list) that contains all the joint names

In [7]:
for i, n in enumerate(robot.model.names):
    print(i, n)

0 universe
1 shoulder_pan_joint
2 shoulder_lift_joint
3 elbow_joint
4 wrist_1_joint
5 wrist_2_joint
6 wrist_3_joint


* robot.model.frames contains all the import frames attached to the robot. 

In [8]:
for f in robot.model.frames:
    print(f.name, 'attached to joint #', f.parent)

universe attached to joint # 0
root_joint attached to joint # 0
world attached to joint # 0
world_joint attached to joint # 0
base_link attached to joint # 0
base_link-base_fixed_joint attached to joint # 0
base attached to joint # 0
shoulder_pan_joint attached to joint # 1
shoulder_link attached to joint # 1
shoulder_lift_joint attached to joint # 2
upper_arm_link attached to joint # 2
elbow_joint attached to joint # 3
forearm_link attached to joint # 3
wrist_1_joint attached to joint # 4
wrist_1_link attached to joint # 4
wrist_2_joint attached to joint # 5
wrist_2_link attached to joint # 5
wrist_3_joint attached to joint # 6
wrist_3_link attached to joint # 6
ee_fixed_joint attached to joint # 6
ee_link attached to joint # 6
wrist_3_link-tool0_fixed_joint attached to joint # 6
tool0 attached to joint # 6


* robot.placement(idx) and robot.framePlacement(idx) returns the placement (i.e. translation+rotation of the joint / frame in argument.

In [12]:
robot.placement(robot.q0, 6)  # Placement of the end effector.

The dimension of the configuration space (i.e. the number of joints) is given in:

In [16]:
NQ = robot.model.nq
NV = robot.model.nv  # for this simple robot, NV == NQ

## Display simple geometries
The robot is displayed in the viewer. We are going to use Meshcat to visualize the 3d robot and scene. First open the viewer and load the robot geometries.

In [9]:
from utils.meshcat_viewer_wrapper import MeshcatVisualizer, colors  # noqa: E402

viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


In [15]:
viz.viewer.jupyter_cell()

As a reminder, a configuration *q* can be displayed in the viewer:

In [13]:
q = np.array([-1., -1.5, 2.1, -.5, -.5, 0])

viz.display(q)

Other geometries (cubes, spheres, etc) can be displayed as well.

In [14]:
# %load lab_0/generated/simple_pick_and_place_2
# Add a red box in the viewer
ballID = "world/ball"
viz.addSphere(ballID, 0.1, colors.red)

# Place the ball at the position ( 0.5, 0.1, 0.2 )
# The viewer expect position and rotation, apppend the identity quaternion
q_ball = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration(ballID, q_ball)


# Forward (direct) geometry

First, let's do some forward geometry, i.e. use Pinocchio to compute where is the end effector knowning the robot configuration. To achieve this we are going to emulate a pick and place scenario (with no physics involved). The red ball we created will be attached to the end effector and thus must move along with it.

# Simple pick ...

Say we have a target at position [.5,.1,.2] and we would like the robot to grasp it.
First decide (by any way you want, e.g. trial and error) the configuration of the robot so that the end effector touches the ball. For that, modify the template code below.


In [19]:
q0 = np.zeros(NQ)  # set the correct values here
q0[0] = 0.5
q0[1] = 0.
q0[2] = -1.5
q0[3] = 0.
q0[4] = 0.
q0[5] = 0.

viz.display(q0)

# Take care to explicitely mention copy when you want a copy of array.
q = q0.copy()

Here is a the solution, should you need it.

In [43]:
# %load lab_0/generated/simple_pick_and_place_3
q0 = np.zeros(NQ)  # set the correct values here
q0[0] = -0.375
q0[1] = -1.2
q0[2] = 1.71
q0[3] = -q0[1] - q0[2]
q0[4] = q0[0]
q0[5] = 0.0

viz.display(q0)
q_init = q0.copy()


# ... and simple place

At the reference position you built, the end effector placement can be obtained by calling 

In [21]:
robot.placement(q, 6).translation

array([ 0.50182315, -0.08022936,  0.19913809])

Only the translation part of the placement has been selected. The rotation is free.

Now, choose any trajectory you want in the configuration space (it can be sinus-cosinus waves, polynomials, splines, straight lines). Make a for loop to display the robot at sampling positions along this trajectory. The function sleep can be used to slow down the loop.

In [22]:
time.sleep(.1)  # in second

At each instant of your loop, recompute the position of the ball and display it so that it always "sticks" to the robot end effector, by modifying the template code below:

In [44]:
# TODO ####################################################
# Replace here with your initial configuration
q0 = np.random.rand(NQ) * 6 - 3
q = q0.copy()
# TODO ####################################################

# Compute initial translation between effector and box.
# Translation of end-eff wrt world at initial configuration
o_eff = robot.placement(q, 6).translation
# Translation of box wrt world
o_ball = q_ball[:3]
eff_ball = o_ball - o_eff

for i in range(10):
    # Replace here by your choice of computing q(t)
    q += np.random.rand(6) * 2e-1 - 1e-1

    # TODO ####################################################
    # Replace here by your computation of the new box position
    o_ball = np.array([0., 0., 0.])
    # /TODO ###################################################

    # Display the new robot and box configurations.
    # The viewer expect a placement (position-rotation).
    viz.applyConfiguration(ballID, o_ball.tolist() + [1, 0, 0, 0])
    viz.display(q)
    time.sleep(0.01)

The solution is below, should you need it.

In [32]:
%do_not_load lab_0/generated/simple_pick_and_place_4

### Pick and place in 6D

Say now that the object is a rectangle and not a sphere. Pick the object at a reference position with the rotation that is imposed, so that the end effector is aligned with one of the faces of the rectangle.


In [60]:
# %load lab_0/generated/simple_pick_and_place_5
# Add a purple box to the viewer
boxID = "world/box"
# viz.delete(ballID)
viz.addBox(boxID, [0.1, 0.2, 0.1], colors.magenta)

# Place the box at the position (0.5, 0.1, 0.2)
q_box = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration(boxID, q_box)
viz.applyConfiguration(ballID, [2, 2, 2, 1, 0, 0, 0])


A configuration with the arm nicely attached to the box is:

In [ ]:
# %load lab_0/generated/simple_pick_and_place_6
q0 = np.zeros(NQ)
q0[0] = -0.375
q0[1] = -1.2
q0[2] = 1.71
q0[3] = -q0[1] - q0[2]
q0[4] = q0[0]

viz.display(q0)
q = q0.copy()


Redo the same question as before, but now also choosing the orientation of the box. For that, at each robot configuration in the for-loop, we will compute the entire box placement with respect the world (let's denote it by oMbox) and display both the box and the robot configuration in the view.

In [66]:
# %load lab_0/generated/simple_pick_and_place_7
# Random velocity of the robot driving the movement
vq = np.array([2., 0, 0, 4., 0, 0])
q = q0.copy()
idx = robot.index('wrist_3_joint')
oMeff = robot.placement(q, idx)  # Placement of end-eff wrt world at current configuration
oMbox = pin.XYZQUATToSE3(q_box)  # Placement of box     wrt world
effMbox = oMeff.inverse() * oMbox  # Placement of box     wrt eff

for i in range(100):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # Gets the new position of the box
    oMbox = robot.placement(q, idx) * effMbox

    # Display new configuration for robot and box
    viz.applyConfiguration(boxID, oMbox)
    viz.display(q)
    time.sleep(1e-2)


## Inverse geometry

As you can see, forward geometry is easy to implement in pinocchio. Let's not see how to deal with inverse geometry.

### Inverse geometry in 3D

Let's now first control the position (i.e. translation only) of the end effector of a manipulator robot to a given position. For this first part, we will use the fixed serial-chain robot model.

Recall first that the position (3D) of the joint with index "i=6" at position "q" can be accessed with:

In [68]:
robot.placement(q, 6).translation

array([ 0.06033124, -0.56198697,  0.57011662])

Using the scipy solver [used in the previous notebook](2_inv_geom.ipynb#section_optim), compute a configuration q where the end effector reaches p. For that, implement a cost function that takes a configuration as argument and returns the squared distance between the end effetor and the target.

In [70]:
%do_not_load lab_0/generated/invgeom3d_1

### Inverse geometry in 6D
Now, let's update the previous cost function for measuring the "distance" between the current placement root.placement(q,6) and a reference placement oMtarget. 
For that, you can use the SE(3) log function to score the distance between two placements. The log returns a 6D velocity, represented by a class Motion, that must be transformed to a vector of R^6 from which you can take the norm.


In [78]:
pin.log(pin.SE3.Identity()).vector

array([0., 0., 0., 0., 0., 0.])

As a target we can define

In [79]:
oMtarget = pin.SE3(pin.utils.rotate("x", 3.14 / 4), np.array([-0.5, 0.1, 0.2]))  # x,y,z

In [101]:
# %load lab_0/generated/invgeom6d_1
# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [0.08] * 3, [0.2, 0.2, 1.0, 0.5])

#
# OPTIM 6D #########################################################
#
def cost(q):
    """Compute score from a configuration"""
    M = robot.placement(q, 6)
    return norm(pin.log(M.inverse() * oMtarget).vector)


def callback(q):
    viz.applyConfiguration(boxID, oMtarget)
    viz.applyConfiguration(tipID, robot.placement(q, 6))
    viz.display(q)
    time.sleep(1e-1)

qopt = fmin_bfgs(cost, robot.q0, callback=callback)

print("The robot finally reached effector placement at\n", robot.placement(qopt, 6))


         Current function value: 0.000004
         Iterations: 34
         Function evaluations: 1612
         Gradient evaluations: 80
The robot finally reached effector placement at
   R =
  0.951459 -0.0480345  0.0161267
-0.0453818   0.676844  -0.648964
 -0.022546   0.655602   0.628759
  p = -0.500001  0.100001  0.200003



/home/stonneau/.local/lib/python3.10/site-packages/scipy/optimize/_optimize.py:1359: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


### Optimizing in the quaternion space

Let's now work with a floating robot: the quadruped solo. This robot has 12 joints, but it's configuration (Q-space) is of dimension 19 (robot.model.nq). Its tangent space (where the velocities are expressed) is of dimension 18 (robot.model.nv). This is because we need 7D vector to encode the robot placement in space (a 3D position and a 4D quaternion).


In [94]:
robot = robex.load('solo12')
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


In [100]:
viz.display(robot.q0)


Run the following code. Can you explain what just happened? Then correct it to have a proper optimization of ANYmal configuration.

In [102]:
# %load lab_0/generated/floating_1
robot.feetIndexes = [
    robot.model.getFrameId(frameName)
    for frameName in ["HR_FOOT", "HL_FOOT", "FR_FOOT", "FL_FOOT"]
]

# --- Add box to represent target
colors = ["red", "blue", "green", "magenta"]
for color in colors:
    viz.addSphere("world/%s" % color, 0.05, color)
    viz.addSphere("world/%s_des" % color, 0.05, color)

#
# OPTIM 6D #########################################################
#

targets = [
    np.array([-0.7, -0.2, 1.2]),
    np.array([-0.3, 0.5, 0.8]),
    np.array([0.3, 0.1, -0.1]),
    np.array([0.9, 0.9, 0.5]),
]
for i in range(4):
    targets[i][2] += 1


def cost(q):
    """Compute score from a configuration"""
    cost = 0.0
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i]).translation
        cost += norm(p_i - targets[i]) ** 2
    return cost


def callback(q):
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i])
        viz.applyConfiguration("world/%s" % colors[i], p_i)
        viz.applyConfiguration(
            "world/%s_des" % colors[i], list(targets[i]) + [1, 0, 0, 0]
        )
    
    viz.display(q)
    time.sleep(1e-2)


Mtarget = pin.SE3(pin.utils.rotate("x", 3.14 / 4), np.array([0.5, 0.1, 0.2]))  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 46
         Function evaluations: 1060
         Gradient evaluations: 53


## Optional Configuration of parallel robots

The following exercise is optional and won't be corrected, but you are free to discuss it on piazza.

A parallel robot is composed of several kinematic chains (called the robot legs) that are all attached to the same end effector. This imposes strict constraints in the configuration space of the robot: a configuration is valide iff all the legs meets the same end-effector placement. We consider here only the geometry aspect of parallel robots (additionnally, some joints are not actuated, which causes additional problems).

The kinematic structure of a parallel robot indeed induces loops in the joint connection graph. In Pinocchio, we can only represent (one of) the underlying kinematic tree. The loop constraints have to be handled separately. An example that loads 4 manipulator arms is given below.

In [103]:
from utils.load_ur5_parallel import load_ur5_parallel  # noqa: E402

robot = load_ur5_parallel()

In [104]:
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/


In [109]:
viz.display(robot.q0)

In [110]:
[w, h, d] = [0.5, 0.5, 0.005]
color = [red, green, blue, transparency] = [1, 1, 0.78, .8]
viz.addBox('world/robot0/toolplate', [w, h, d], color)
Mtool = pin.SE3(pin.utils.rotate('z', 1.268), np.array([0, 0, .75]))
viz.applyConfiguration('world/robot0/toolplate', Mtool)

The 4 legs of the robot are loaded in a single robot model. The 4 effector placements are computed by:

In [111]:
effIdxs = [robot.model.getFrameId('tool0_#%d' % i) for i in range(4)]
robot.framePlacement(robot.q0, effIdxs[0])

  R =
   -0.707107    -0.707107 -3.46245e-12
    0.707107    -0.707107 -3.46236e-12
-6.12323e-17 -4.89664e-12            1
  p = 0.306122 0.160483 0.749342

The loop constraints are that the relative placement of every leg end-effector must stay the same as it was in the initial configuration (at the configuration *robot.q0*) and the plate placement *Mtool*. To be valid, a configuration *q* must satisfy these 4 relative placement constraints.

Consider now that the orientation of the tool plate is given by the following quaternion, with the translation that you like (see [the notebook about rotations if you need more details](appendix_quaternions.ipynb)): 



In [108]:
quat = pin.Quaternion(0.7, 0.2, 0.2, 0.6).normalized()
print(quat.matrix())

[[ 0.13978495 -0.8172043   0.55913978]
 [ 0.98924731  0.13978495 -0.04301075]
 [-0.04301075  0.55913978  0.82795699]]


**Find using the above optimization routines the configuration of each robot leg so that the loop constraints are all met** for the new orientation of the plate.